In [ ]:
# MO444-A 2s/2017 - Second assignment
#
#         Group 05
#
# - Anderson Rossanez (124136)
# - Bruno Branta Lopes (31470)
#

from __future__ import print_function
import glob
import random
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
from skimage.io import imread
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Some definitions
classes = np.asarray(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
int_classes = preprocessing.LabelEncoder().fit_transform(classes) # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
onehot_classes = onehot_encoder.fit_transform(int_classes.reshape(len(int_classes), 1))

def load_image_dataset(name, sample=1., as_gray=False):
    """Loads an image dataset
    name: directory in which dataset is defined
    sample: fraction of the dataset to be loaded [0, 1]
    as_gray: read images as grayscale to have a 32x32 matrix, instead of a 32x32x3 matrix in case of RGB.
    """
    random.seed(1)
    X, Y = [], []
    with open('cifar-10/%s/labels' % name) as labels:
        i = 0
        for path in sorted(glob.glob('cifar-10/%s/*.png' % name)):
            y = int(labels.next())
            if random.random() > sample:
                continue
            Y.append(y)
            X.append(imread(path, as_grey=as_gray))
    X, Y = np.array(X), np.array(Y)
    n_dim = reduce(lambda x, y: x * y, X.shape[1:])
    return X.reshape(-1, n_dim), Y

def display_image(x):
    """Prints a colored or grayscale 32x32 image"""
    colors = x.shape[0] / 32 / 32
    cmap = 'gray' if colors == 1 else 'jet'
    new_shape = (32, 32) if colors == 1 else (32, 32, colors)
    plt.imshow(x.reshape(new_shape), cmap=cmap)
    plt.show()


In [ ]:
# Load train dataset
data_X, data_Y = load_image_dataset('train', as_gray=True, sample=0.1)
print("Loaded %d samples" % len(data_X))

In [ ]:
# Take a look at a sample and its class
display_image(data_X[207])

print('Label: %s' % classes[int(data_Y[207])])

In [ ]:
# Implementing K-Fold to help avoiding overfitting
from sklearn.model_selection import KFold
from datetime import datetime

random_state = np.random.RandomState(1)

# prepare for 5-fold execution
k5_fold = KFold(n_splits=5, shuffle=True, random_state=random_state)

def run_kfold(method, data_Y, data_X, scale=False):
    train_precision, train_recall, train_f1 = [], [], []
    validation_precision, validation_recall, validation_f1 = [], [], []
    start_time = datetime.now()
    k = 0
    print('k', end=' ')
    model = None

    for train_index, validation_index in k5_fold.split(data_X):
        k += 1
        print(k, end=' ')
        
        train_data_X, train_data_Y = data_X[train_index], data_Y[train_index]
        validation_data_X, validation_data_Y = data_X[validation_index], data_Y[validation_index]
        
        if scale:
            model_scaler = preprocessing.StandardScaler()
            train_data_X = model_scaler.fit_transform(train_data_X)
            validation_data_X = model_scaler.transform(validation_data_X)

        # Train the model(s) using the training data
        model = method(train_data_X, train_data_Y)
        
        # Predict training data
        predicted_train_data_Y = model.predict(train_data_X)
        train_precision.append(precision_score(train_data_Y, predicted_train_data_Y, average='weighted'))
        train_recall.append(recall_score(train_data_Y, predicted_train_data_Y, average='weighted'))
        train_f1.append(f1_score(train_data_Y, predicted_train_data_Y, average='weighted'))
        
        # Predict validation data
        predicted_validation_data_Y = model.predict(validation_data_X)
        validation_precision.append(precision_score(validation_data_Y, predicted_validation_data_Y, average='weighted'))
        validation_recall.append(recall_score(validation_data_Y, predicted_validation_data_Y, average='weighted'))
        validation_f1.append(f1_score(validation_data_Y, predicted_validation_data_Y, average='weighted'))
    
    print('time elapsed: {}\n'.format(datetime.now() - start_time))
    print('           Precision  sd      Recall sd     F1 Score sd')
    print('Training:      %5.2f  ±%5.2f   %5.2f ±%5.2f    %5.2f ±%5.2f' % (
            np.mean(train_precision), np.std(train_precision), np.mean(train_recall), np.std(train_recall), np.mean(train_f1), np.std(train_f1)))
    print('Validation:    %5.2f  ±%5.2f   %5.2f ±%5.2f    %5.2f ±%5.2f' % (
            np.mean(validation_precision), np.std(validation_precision), np.mean(validation_recall), np.std(validation_recall), np.mean(validation_f1), np.std(validation_f1)))
    return model

In [ ]:
# Create a baseline One-vs-All logistic regression model
ova_lr_model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=100)
run_kfold(lambda X, Y: ova_lr_model.fit(X, Y), data_Y, data_X, scale=True)

In [ ]:
# Create a Multinomial (Softmax) logistic regression model
mn_lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=100)
run_kfold(lambda X, Y: mn_lr_model.fit(X, Y), data_Y, data_X, scale=True)

In [ ]:
# Neural Networks
class NeuralNetwork(object):
    
    def __init__(self, input_size, output_size, learning_rate=0.5, epochs=10, batch_size=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
        # input data placeholders
        self.x = tf.placeholder(tf.float32, [None, input_size])
        # output data placeholder
        self.y = tf.placeholder(tf.float32, [None, output_size])

        # now declare the weights connecting the input to the hidden layer
        self.W1 = tf.Variable(tf.random_normal([input_size, 300], stddev=0.03), name='W1')
        b1 = tf.Variable(tf.random_normal([300]), name='b1')

        # and the weights connecting the hidden layer to the output layer
        self.W2 = tf.Variable(tf.random_normal([300, output_size], stddev=0.03), name='W2')
        b2 = tf.Variable(tf.random_normal([output_size]), name='b2')
        
        # hidden layer's output
        Z1 = tf.add(tf.matmul(self.x, self.W1), b1)
        a1 = self.activation(Z1)
        
        # output with softmax activation
        Z2 = tf.add(tf.matmul(a1, self.W2), b2)
        a2 = self.activation(Z2)
        self.y_pred = tf.nn.softmax(a2)
        
        # Cost function
        y_pred_clipped = tf.clip_by_value(self.y_pred, 1e-10, 0.9999999) # To avoid log(0), returning NaN
        self.cost = -tf.reduce_mean(tf.reduce_sum(self.y * tf.log(y_pred_clipped) + (1 - self.y) * tf.log(1 - y_pred_clipped), axis=1))
        
        # Optimizer
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        # Gradient checking
        #self.gradients = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).compute_gradients(self.cost, self.get_trainable_variables())
        
        # finally setup the initialisation operator
        self.init_op = tf.global_variables_initializer()
        
        # define a tensorflow session
        self.session = tf.Session()
        self.session.run(self.init_op)

    def activation(self, x):
        return tf.div(tf.constant(1.0), tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))
    
    def activation_prime(self, x):
        return tf.multiply(activation(x), tf.subtract(tf.constant(1.0), activation(x)))
        
    #def get_trainable_variables(self):
        #return np.concatenate(self.W1, self.W2)

    def fit(self, X, Y):
        # Apply One Hot Encoding
        Y = onehot_encoder.transform(Y.reshape(len(Y), 1))
        
        costs = []
        # Training
        for epoch in range(self.epochs):
            _, c = self.session.run([self.optimizer, self.cost], feed_dict={self.x: X, self.y: Y})
            # print("Epoch:", (epoch + 1), "cost: ", c)
            costs.append(c)
            #if epoch == 5:
            #    gradients = self.session.run(self.gradients)

        self.print_costs(costs)
        return self
    
    def print_costs(self, costs):
        plt.plot(costs)
        plt.ylabel('cost')
        plt.xlabel('epoch')
        plt.show()

    def predict(self, X):
        # Forward propagation
        prediction = self.session.run(self.y_pred, feed_dict={self.x: X})
        return np.argmax(prediction, 1)

In [ ]:
# Neural Network with a single hidden layer
nn_single_hl_model = NeuralNetwork(input_size=len(data_X[0]), output_size=len(classes))
run_kfold(lambda X, Y: nn_single_hl_model.fit(X, Y), data_Y, data_X)